In [1]:
import numpy as np
from PIL import Image, ImageDraw


In [2]:
# Function to generate symmetric images
def generate_symmetric_image(img_size=(200, 200)):
    img = Image.new('RGB', img_size, (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Generate random control points for one half
    control_points = [(np.random.randint(0, img_size[0] // 2), np.random.randint(0, img_size[1])) for _ in range(5)]
    
    # Connect the points to form a polygon
    draw.polygon(control_points, outline=(0, 0, 0))

    # Mirror the polygon to the other half
    mirror_points = [(img_size[0] - x, y) for x, y in control_points]
    draw.polygon(mirror_points, outline=(0, 0, 0))

    return img


In [3]:
# Function to generate asymmetric images
def generate_asymmetric_image(img_size=(200, 200)):
    img = Image.new('RGB', img_size, (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Generate random control points for both halves
    control_points_left = [(np.random.randint(0, img_size[0] // 2), np.random.randint(0, img_size[1])) for _ in range(5)]
    control_points_right = [(np.random.randint(img_size[0] // 2, img_size[0]), np.random.randint(0, img_size[1])) for _ in range(5)]
    
    # Draw two different polygons for each half
    draw.polygon(control_points_left, outline=(0, 0, 0))
    draw.polygon(control_points_right, outline=(0, 0, 0))

    return img

In [4]:
# Example usage:
symmetric_img = generate_symmetric_image()
asymmetric_img = generate_asymmetric_image()

In [5]:

symmetric_img.save('/Users/lykofos/Desktop/study/SEM-3/CS786/symmetric_example.png')
asymmetric_img.save('/Users/lykofos/Desktop/study/SEM-3/CS786/asymmetric_example.png')

In [6]:
for i in range(8000):
    img = generate_symmetric_image()
    img.save(f'dataset/train/symmetric/sym_{i}.png')

for i in range(8000):
    img = generate_asymmetric_image()
    img.save(f'dataset/train/asymmetric/asym_{i}.png')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lykofos/Desktop/study/SEM-3/CS786/dataset/train/symmetric/sym_0.png'

In [7]:
#saving the dataset
import os
# Define base path to save images
base_path = '/Users/lykofos/Desktop/study/SEM-3/CS786/'

# Create necessary directories 
def create_directories():
    for subset in ['train', 'val', 'test']:
        os.makedirs(os.path.join(base_path, subset, 'symmetric'), exist_ok=True)
        os.makedirs(os.path.join(base_path, subset, 'asymmetric'), exist_ok=True)

# Function to generate symmetric images
def generate_symmetric_image(img_size=(200, 200)):
    img = Image.new('RGB', img_size, (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Generate random control points for one half
    control_points = [(np.random.randint(0, img_size[0] // 2), np.random.randint(0, img_size[1])) for _ in range(5)]
    
    # Connect the points to form a polygon
    draw.polygon(control_points, outline=(0, 0, 0))

    # Mirror the polygon to the other half
    mirror_points = [(img_size[0] - x, y) for x, y in control_points]
    draw.polygon(mirror_points, outline=(0, 0, 0))

    return img

# Function to generate asymmetric images
def generate_asymmetric_image(img_size=(200, 200)):
    img = Image.new('RGB', img_size, (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Generate random control points for both halves
    control_points_left = [(np.random.randint(0, img_size[0] // 2), np.random.randint(0, img_size[1])) for _ in range(5)]
    control_points_right = [(np.random.randint(img_size[0] // 2, img_size[0]), np.random.randint(0, img_size[1])) for _ in range(5)]
    
    # Draw two different polygons for each half
    draw.polygon(control_points_left, outline=(0, 0, 0))
    draw.polygon(control_points_right, outline=(0, 0, 0))

    return img

# Function to generate and save dataset
def generate_dataset(num_images_per_class=1000):
    for subset in ['train', 'val', 'test']:
        for i in range(num_images_per_class):
            # Generate symmetric and asymmetric images
            sym_img = generate_symmetric_image()
            asym_img = generate_asymmetric_image()

            # Save the images in the respective directories
            sym_img.save(os.path.join(base_path, subset, 'symmetric', f'sym_{i}.png'))
            asym_img.save(os.path.join(base_path, subset, 'asymmetric', f'asym_{i}.png'))

# Create directories
create_directories()

# Generate and save 1000 images for train, 200 for validation, and 200 for test
generate_dataset(num_images_per_class=1000)  # Adjust this for val and test

print("Dataset generation complete.")

Dataset generation complete.


In [8]:
import tensorflow as tf

# Load the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/lykofos/Desktop/study/SEM-3/CS786/train', 
    image_size=(200, 200), 
    batch_size=32, 
    label_mode='binary'
)

# Load the validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/lykofos/Desktop/study/SEM-3/CS786/val', 
    image_size=(200, 200), 
    batch_size=32, 
    label_mode='binary'
)

# Load the test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/lykofos/Desktop/study/SEM-3/CS786/test', 
    image_size=(200, 200), 
    batch_size=32, 
    label_mode='binary'
)

Found 2000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [11]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Initialize model without pre-trained weights
base_model = InceptionResNetV2(weights=None, include_top=False, input_shape=(200, 200, 3))

# Add new layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
import tensorflow as tf

# Define the dataset path
dataset_path = '/Users/lykofos/Desktop/study/SEM-3/CS786/train'

# Load training and validation data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path, 
    image_size=(200, 200), 
    batch_size=32, 
    label_mode='binary'
)

# Train the model
history = model.fit(train_dataset, epochs=10)

Found 2000 files belonging to 2 classes.
Epoch 1/10
63/63 [==============================] - 998s 15s/step - loss: 0.5530 - accuracy: 0.7765
Epoch 2/10
63/63 [==============================] - 1154s 18s/step - loss: 0.1229 - accuracy: 0.9605
Epoch 3/10
63/63 [==============================] - 1727s 28s/step - loss: 0.0789 - accuracy: 0.9770
Epoch 4/10
63/63 [==============================] - 1148s 18s/step - loss: 0.0537 - accuracy: 0.9825
Epoch 5/10
63/63 [==============================] - 906s 14s/step - loss: 0.0379 - accuracy: 0.9900
Epoch 6/10
63/63 [==============================] - 933s 15s/step - loss: 0.0014 - accuracy: 1.0000
Epoch 7/10
63/63 [==============================] - 978s 16s/step - loss: 2.4617e-04 - accuracy: 1.0000
Epoch 8/10
63/63 [==============================] - 1191s 19s/step - loss: 0.0440 - accuracy: 0.9880
Epoch 9/10
63/63 [==============================] - 837s 13s/step - loss: 0.0480 - accuracy: 0.9830
Epoch 10/10
63/63 [==============================] 

In [13]:
# Define the path to the test set
test_dataset_path = '/Users/lykofos/Desktop/study/SEM-3/CS786/test'

# Load the test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    image_size=(200, 200),
    batch_size=32,
    label_mode='binary'
)

# Evaluate the model on the basic test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Round 1 - Basic Testing: Test Accuracy = {test_acc:.2f}, Test Loss = {test_loss:.2f}")

Found 2000 files belonging to 2 classes.
63/63 [==============================] - 202s 3s/step - loss: 0.0541 - accuracy: 0.9845
Round 1 - Basic Testing: Test Accuracy = 0.98, Test Loss = 0.05


In [14]:
import os

# Modify shapes slightly in symmetric images
def modify_symmetric_images(img):
    draw = ImageDraw.Draw(img)
    # Add a small shape randomly within the symmetric image
    x = np.random.randint(10, 190)
    y = np.random.randint(10, 190)
    r = np.random.randint(5, 15)
    draw.ellipse((x - r, y - r, x + r, y + r), fill="black")
    return img

# Add small random shapes to asymmetric images
def modify_asymmetric_images(img):
    draw = ImageDraw.Draw(img)
    # Add 2-3 small shapes randomly
    for _ in range(3):
        x = np.random.randint(10, 190)
        y = np.random.randint(10, 190)
        r = np.random.randint(5, 15)
        draw.rectangle((x - r, y - r, x + r, y + r), fill="black")
    return img

In [15]:
def generate_modified_dataset(round_num, modification_function):
    # Paths for modified datasets
    modified_path = f'/Users/lykofos/Desktop/study/SEM-3/CS786/round_{round_num}'
    os.makedirs(modified_path, exist_ok=True)
    os.makedirs(f"{modified_path}/symmetric", exist_ok=True)
    os.makedirs(f"{modified_path}/asymmetric", exist_ok=True)

    # Modify symmetric images
    for img_name in os.listdir(test_dataset_path + "/symmetric"):
        img = Image.open(os.path.join(test_dataset_path, "symmetric", img_name))
        modified_img = modification_function(img)
        modified_img.save(os.path.join(modified_path, "symmetric", img_name))

    # Modify asymmetric images
    for img_name in os.listdir(test_dataset_path + "/asymmetric"):
        img = Image.open(os.path.join(test_dataset_path, "asymmetric", img_name))
        modified_img = modification_function(img)
        modified_img.save(os.path.join(modified_path, "asymmetric", img_name))

# Generate datasets for Rounds 2, 3, and 4
generate_modified_dataset(2, modify_symmetric_images)  # Round 2
generate_modified_dataset(3, modify_asymmetric_images)  # Round 3
generate_modified_dataset(4, lambda img: modify_asymmetric_images(modify_symmetric_images(img)))  # Round 4

In [16]:
for round_num in range(1, 5):
    if round_num == 1:
        dataset_path = test_dataset_path  # Basic test set for Round 1
    else:
        dataset_path = f'/Users/lykofos/Desktop/study/SEM-3/CS786/round_{round_num}'  # Distorted sets for Rounds 2-4

    # Load the dataset for the current round
    round_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        image_size=(200, 200),
        batch_size=32,
        label_mode='binary'
    )

    # Evaluate the model
    round_loss, round_acc = model.evaluate(round_dataset)
    print(f"Round {round_num} - Deliberate Test: Test Accuracy = {round_acc:.2f}, Test Loss = {round_loss:.2f}")

Found 2000 files belonging to 2 classes.
63/63 [==============================] - 198s 3s/step - loss: 0.0541 - accuracy: 0.9845
Round 1 - Deliberate Test: Test Accuracy = 0.98, Test Loss = 0.05
Found 2000 files belonging to 2 classes.
63/63 [==============================] - 198s 3s/step - loss: 0.5194 - accuracy: 0.8675
Round 2 - Deliberate Test: Test Accuracy = 0.87, Test Loss = 0.52
Found 2000 files belonging to 2 classes.
63/63 [==============================] - 196s 3s/step - loss: 1.2822 - accuracy: 0.7360
Round 3 - Deliberate Test: Test Accuracy = 0.74, Test Loss = 1.28
Found 2000 files belonging to 2 classes.
63/63 [==============================] - 196s 3s/step - loss: 1.5950 - accuracy: 0.6580
Round 4 - Deliberate Test: Test Accuracy = 0.66, Test Loss = 1.60


In [17]:
from sklearn.metrics import precision_score, recall_score

# Evaluate each round and record metrics
results = []
for round_num in range(1, 5):
    dataset_path = test_dataset_path if round_num == 1 else f'/Users/lykofos/Desktop/study/SEM-3/CS786/round_{round_num}'

    # Load dataset and evaluate model predictions
    round_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        image_size=(200, 200),
        batch_size=32,
        label_mode='binary'
    )
    
    y_true = []
    y_pred = []
    for images, labels in round_dataset:
        predictions = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend((predictions > 0.5).astype(int).flatten())
    
    # Calculate metrics
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    results.append({
        'Round': round_num,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall
    })

# Display results
for result in results:
    print(f"Round {result['Round']} - Accuracy: {result['Accuracy']:.2f}, Precision: {result['Precision']:.2f}, Recall: {result['Recall']:.2f}")

Found 2000 files belonging to 2 classes.
1/1 [==============================] - 4s 4s/step
Found 2000 files belonging to 2 classes.
1/1 [==============================] - 3s 3s/step
Found 2000 files belonging to 2 classes.
1/1 [==============================] - 2s 2s/step
Found 2000 files belonging to 2 classes.
1/1 [==============================] - 2s 2s/step
Round 1 - Accuracy: 0.50, Precision: 0.97, Recall: 1.00
Round 2 - Accuracy: 0.50, Precision: 0.95, Recall: 0.78
Round 3 - Accuracy: 0.50, Precision: 0.90, Recall: 0.53
Round 4 - Accuracy: 0.50, Precision: 0.81, Recall: 0.42
